In [1]:
import pandas as pd
import difflib

In [2]:
dan = pd.read_csv("NetflixViewingHistoryDan.csv")
netflix = pd.read_csv("netflix_titles.csv")

In [3]:
bestMatches = dan['Title'].apply(lambda x: difflib.get_close_matches(x, netflix['title']))
bestMatches

0                    [Big Time Rush, Big Time, Big Kill]
1      [Monster Island, Monster House, Super Monsters...
2                                                     []
3                              [Brand New Cherry Flavor]
4      [Starship Troopers: Traitor of Mars, Star Trek...
                             ...                        
973                                         [Battlefish]
974                         [Blackfish, Black, Backfire]
975               [The Emigrant, The Giant, The Reliant]
976                                    [Elisa & Marcela]
977                        [Red Dawn, Red Joan, Red Dot]
Name: Title, Length: 978, dtype: object

In [4]:
genres = []
for i in range(len(bestMatches)):
    itemGenres = []
    for j in range(len(bestMatches[i])):
        itemGenres.append(netflix['listed_in'].values[netflix['title'] == bestMatches[i][j]])
    genres.append(itemGenres)
genres

[[array(["Kids' TV"], dtype=object),
  array(['Documentaries'], dtype=object),
  array(['Action & Adventure'], dtype=object)],
 [array(['Children & Family Movies'], dtype=object),
  array(['Children & Family Movies, Comedies'], dtype=object),
  array(['Children & Family Movies'], dtype=object)],
 [],
 [array(['TV Dramas, TV Horror, TV Mysteries'], dtype=object)],
 [array(['Action & Adventure, Sci-Fi & Fantasy'], dtype=object),
  array(['Action & Adventure, Sci-Fi & Fantasy'], dtype=object),
  array(["Kids' TV"], dtype=object)],
 [array(['TV Comedies'], dtype=object)],
 [],
 [array(['TV Comedies'], dtype=object)],
 [],
 [],
 [],
 [],
 [],
 [array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object)],
 [array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object)],
 [array(['

In [5]:
finalGenres = []
for i in range(len(genres)):
    for j in range(len(genres[i])):
        for k in range(len(genres[i][j])):
            finalGenres += list(genres[i][j])

finalGenres

["Kids' TV",
 'Documentaries',
 'Action & Adventure',
 'Children & Family Movies',
 'Children & Family Movies, Comedies',
 'Children & Family Movies',
 'TV Dramas, TV Horror, TV Mysteries',
 'Action & Adventure, Sci-Fi & Fantasy',
 'Action & Adventure, Sci-Fi & Fantasy',
 "Kids' TV",
 'TV Comedies',
 'TV Comedies',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'Action & Adventure, Sci-Fi & Fantasy',
 'Dramas',
 "Kids' TV, TV Comedies",
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British TV Shows, Reality TV',
 'British T

In [6]:
genreCounts = pd.Series(finalGenres).value_counts()
genreCounts

Stand-Up Comedy                                      88
Action & Adventure                                   48
TV Comedies                                          39
Docuseries                                           34
British TV Shows, Reality TV                         33
                                                     ..
Action & Adventure, Cult Movies, Sci-Fi & Fantasy     1
Classic Movies, Cult Movies, Dramas                   1
Music & Musicals                                      1
Classic Movies, Documentaries                         1
Action & Adventure, Cult Movies                       1
Length: 179, dtype: int64

In [7]:
listOfGenres = list(genreCounts.keys())
listOfGenres

['Stand-Up Comedy',
 'Action & Adventure',
 'TV Comedies',
 'Docuseries',
 'British TV Shows, Reality TV',
 'Dramas',
 'Documentaries',
 'Action & Adventure, Comedies',
 'Dramas, International Movies',
 'Children & Family Movies, Comedies',
 'Thrillers',
 'Action & Adventure, Dramas, International Movies',
 'Action & Adventure, International Movies',
 'Dramas, Independent Movies, International Movies',
 'Crime TV Shows, International TV Shows, TV Dramas',
 'Action & Adventure, Sci-Fi & Fantasy',
 'Comedies, Dramas, International Movies',
 'Comedies',
 "Kids' TV, TV Comedies",
 'International TV Shows, TV Dramas',
 'Movies',
 "Kids' TV",
 'Action & Adventure, Dramas',
 'Dramas, Independent Movies',
 'Documentaries, International Movies',
 'Dramas, Thrillers',
 'Crime TV Shows, TV Action & Adventure, TV Thrillers',
 'Crime TV Shows, Docuseries',
 'Horror Movies, Thrillers',
 'Dramas, International Movies, Thrillers',
 'Action & Adventure, Children & Family Movies, Classic Movies',
 'Dram

In [8]:
genresAndMovies = {}
for i in range(netflix.shape[0]):
    if not genresAndMovies.get(netflix.iloc[i]['listed_in']):
        genresAndMovies[netflix.iloc[i]['listed_in']] = []
    genresAndMovies[netflix.iloc[i]['listed_in']].append(netflix.iloc[i]['title'])
genresAndMovies

{'Documentaries': ['Dick Johnson Is Dead',
  'My Heroes Were Cowboys',
  'Final Account',
  'Bob Ross: Happy Accidents, Betrayal & Greed',
  'Lady Boss: The Jackie Collins Story',
  'Fantastic Fungi',
  '9to5: The Story of a Movement',
  'This Changes Everything',
  'FTA',
  'Headspace: Unwind Your Mind',
  'Sir! No Sir!',
  'The Reason I Jump',
  'Picture a Scientist',
  'Breaking Boundaries: The Science Of Our Planet',
  'Black Holes | The Edge of All We Know',
  'Dream/Killer',
  'Creating an Army of the Dead',
  'The Last Days',
  'Amy Tan: Unintended Memoir',
  "In Our Mothers' Gardens",
  'Unrest',
  'Free to Play',
  'Why Did You Kill Me?',
  'Coded Bias',
  'White Boy',
  '7 Yards: The Chris Norton Story',
  'Elizabeth and Margaret: Love and Loyalty',
  'Seaspiracy',
  'Operation Varsity Blues:\xa0The\xa0College\xa0Admissions\xa0Scandal',
  'The Last Blockbuster',
  'Audrey',
  'The Most Unknown',
  'Strip Down, Rise Up',
  'What Would Sophia Loren Do?',
  'Crack: Cocaine, Corr

In [9]:
closeMovies = {}
for i in range(10):
    key = listOfGenres[i]
    movies = genresAndMovies.get(key)
    closeMovies[key] = []
    listOfLists = list(dan["Title"].apply(lambda x: difflib.get_close_matches(x, movies)))
    for subList in listOfLists:
        closeMovies[key] += subList


In [10]:
for key, value in closeMovies.items():
    print(key)
    print(list(pd.Series(value).value_counts().keys())[:20])
    print()

Stand-Up Comedy
['Jim Gaffigan: Obsessed', 'Jim Gaffigan: Cinco', 'Kevin Hart: Laugh at My Pain', 'Jim Jefferies: Intolerant', 'Jim Jefferies: Freedumb', 'Jim Gaffigan: King Baby', 'Hannibal Buress: Comedy Camisado', 'Trevor Noah: Son of Patricia', 'Patton Oswalt: Talking for Clapping', 'Jim Jefferies : BARE', 'Tom Segura: Completely Normal', 'Tom Segura: Mostly Stories', 'Bert Kreischer: Hey Big Boy', 'Trevor Noah: Afraid of the Dark', 'Kevin Hart: What Now?', 'Kevin Hart: Let Me Explain', 'Jim Gaffigan: Mr. Universe', 'Patton Oswalt: Annihilation', 'Tom Segura: Ball Hog', 'Bert Kreischer: The Machine']

Action & Adventure
['The Losers', 'The Saint', 'The Prince', 'The Art of War', 'The Peacemaker', 'Ninja Assassin', 'American Assassin', 'Kill Bill: Vol. 2', 'Olympus Has Fallen', 'Angel Has Fallen', 'Act of Valor', 'The Old Guard', 'The Hard Way', 'The Long Riders', 'The 2nd', 'Kill Bill: Vol. 1', 'Takers', 'Casino Royale', 'Pusher', 'Troy: The Odyssey']

TV Comedies
['The Office (U.S